In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
import torch
import torch as T
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

RANDOM_SEED = 42  # Answer to the Ultimate Question of Life, the Universe, and Everything
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)
        # Consider adding another layer
        self.hidden_2 = torch.nn.Linear(n_hidden, n_hidden)
        self.predict = torch.nn.Linear(n_hidden, n_output)

    def forward(self, x):
        x = F.relu(self.hidden(x))
        x = F.relu(self.hidden_2(x))
        x = self.predict(x)
        return x


def train_network(train_x, train_y, net):
    net = net.train()  # set training mode
    loss_func = T.nn.MSELoss()  # mean squared error
    # loss = torch.sqrt(criterion(x, y))

    optimizer = T.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    X = train_x
    Y = train_y
    for b in range(10):
        optimizer.zero_grad()
        oupt = net(X.float())
        loss_obj = torch.sqrt(loss_func(oupt, Y.float()))  # Root mean squared error
        loss_obj.backward()
        optimizer.step()
    return net

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/Causal_Inference_Final

/content/drive/MyDrive/Colab Notebooks/Causal_Inference_Final


In [ ]:
!ls ./data

boston.csv  concrete.xls  energy.xlsx


In [ ]:
 def dataset_prep(df):
    label_column = df.columns[-1]
    train, test = train_test_split(df, test_size=0.1)

    x_train = train.loc[:, train.columns != label_column]
    y_train = train.loc[:, train.columns == label_column]

    x_test = test.loc[:, test.columns != label_column]
    y_test = test.loc[:, test.columns == label_column]
    return torch.from_numpy(x_train.values).float(), torch.from_numpy(y_train.values).float(), torch.from_numpy(x_test.values).float(), torch.from_numpy(y_test.values).float()

In [ ]:
class RegressionDataset(Dataset):
    def __init__(self, x_df, y_df):
        # for batch, (x, y) in enumerate(dataloader):
        self.x = x_df
        self.y = y_df 
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx): 
        return self.x[idx], self.y[idx]


In [ ]:
def data_loader_pref(df):
    x_train, y_train, x_test, y_test = dataset_prep(df)
    train_dataset = RegressionDataset(x_train, y_train)
    trainloader = DataLoader(train_dataset, batch_size=16)
    test_dataset = RegressionDataset(x_test, y_test)
    testloader = DataLoader(test_dataset, batch_size=16)
    return trainloader, testloader, len(train_dataset[0][0])

In [ ]:
def train_linear_regressor(train_loader, test_loader, n_dimensions):
    net = Net(n_feature=n_dimensions, n_hidden=64, n_output=1)
    net = net.train()  # set training mode
    loss_func = T.nn.MSELoss()  # mean squared error
    # loss = torch.sqrt(criterion(x, y))

    optimizer = T.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
  
    for epoch in range(10):
        for batch, (x, y) in enumerate(train_loader):
            optimizer.zero_grad()
            oupt = net(x)
            loss_obj = torch.sqrt(loss_func(oupt, y))  # Root mean squared error
            loss_obj.backward()
            optimizer.step()
    total_loss = 0
    for batch, (x, y) in enumerate(test_loader):
        out = net(x)
        loss_obj = torch.sqrt(loss_func(out, y))
        total_loss += loss_obj.item()

    return total_loss/len(test_loader)


In [ ]:
boston_df = pd.read_csv("./data/boston.csv",header=None, delim_whitespace=True)

In [ ]:
concrete_ = pd.read_excel("./data/concrete.xls")

In [ ]:
energy = pd.read_excel("./data/energy.xlsx")

In [ ]:
def linear_regressor_wrapper(df):
    train_loader, test_loader, n_dimensions = data_loader_pref(df)
    eval_loss = train_linear_regressor(train_loader, test_loader, n_dimensions)
    return eval_loss

In [ ]:
boston_loss = linear_regressor_wrapper(boston_df)
print('Boston RMSE Loss: {}'.format(boston_loss))
energy_loss = linear_regressor_wrapper(energy)
print('Energy RMSE Loss: {}'.format(energy_loss))
concrete_loss = linear_regressor_wrapper(concrete_)
print('Concrete RMSE Loss: {}'.format(concrete_loss))


Boston RMSE Loss: 19.565793991088867
Energy RMSE Loss: 22.410636520385744
Concrete RMSE Loss: 37.00589043753488


In [ ]:
class Latent_Confounder_Encoder(nn.Module):
    def __init__(self, n_dimensions, output_dimension, hidden_size=64):
        super().__init__()
        self.first_net = Net(n_dimensions, hidden_size, output_dimension)
        self.second_net =Net(n_dimensions, hidden_size, output_dimension)

        self.normal = torch.distributions.Normal
    def forward(self, z):
        first_out = self.first_net(z)
        second_out = self.second_net(z)

        return self.normal(first_out, second_out)


class Encoder_Decoder_Model(nn.Module):
    def __init__(self, n_dimensions, output_dimension, hidden_size=64):
        super().__init__()
        self.first_net = Net(n_dimensions, hidden_size, n_dimensions)
        self.second_net = Net(n_dimensions, hidden_size, output_dimension)
        self.third_net = Net(n_dimensions, hidden_size, output_dimension)

        self.normal = torch.distributions.Normal

    def forward(self,x, y=None):
        if y != None:
            x = torch.cat((x, y), dim=1)
        # N(z| g 6 ∘ g 5 ([x, t]), g 7 ∘ g 5 ([x, t]) )
        # N(y| g 2 ∘ g 1 ([z, t]), g 3 ∘ g 1 ([z, t]) )
        common_out = self.first_net(x)
        return self.normal(self.second_net(common_out), self.third_net(common_out))

        #      I guess this will be Z 


In [ ]:
class GenerativeModel(nn.Module):
    def __init__(self,n_dimensions, hidden_size=64):
        super().__init__()

        self.decoder_y = Encoder_Decoder_Model(n_dimensions=n_dimensions+1,
                                               output_dimension=1,
                                               hidden_size=hidden_size)
        
        self.latent_x = Latent_Confounder_Encoder(n_dimensions=n_dimensions,
                                                  output_dimension=n_dimensions)
        
        self.latent_t = Latent_Confounder_Encoder(n_dimensions=n_dimensions, 
                                                  output_dimension=1)


    def forward(self, z, t):
        x_encoded = self.latent_x(z) # Why are this necessary ?

        t_encoded = self.latent_t(z) # Why are this necessary ?

        y_decoded = self.decoder_y(z, t)

        return y_decoded, t_encoded, x_encoded

class InferenceModel(nn.Module):
    def __init__(self,n_dimensions, hidden_size=64):
        super().__init__()
        self.encoder_z = Encoder_Decoder_Model(n_dimensions=n_dimensions,
                                               output_dimension=n_dimensions-1, 
                                               hidden_size=hidden_size)
        # q(z|x, t) = N(z| g 6 ∘ g 5 ([x, t]), g 7 ∘ g 5 ([x, t]) )
    

    def forward(self, x, t):
        z_encoded = self.encoder_z(x, t)

        return z_encoded
        

class VAE(nn.Module):
    def __init__(self,n_dimensions, hidden_size=64):
        super().__init__()


        self.generative = GenerativeModel(n_dimensions=n_dimensions, 
                                        hidden_size=hidden_size)
        self.inference = InferenceModel(n_dimensions=n_dimensions+1,
                                    hidden_size=hidden_size)
    def forward(self,x, t):
        z_decoded = self.inference(x, t)

        y_decoded, t_encoded, x_encoded = self.generative(z_decoded.sample(), t)

        return z_decoded, y_decoded, t_encoded, x_encoded

        


In [ ]:
def log_norm(x, mu, std):
    """Compute the log pdf of x,
    under a normal distribution with mean mu and standard deviation std."""
    
    return torch.mean(-0.5 * torch.log(2*np.pi*std**2) -(0.5 * (1/(std**2))* (x-mu)**2))

def elbo_loss(y_decoded, y, z_decoded, prior_z):
    # Fix this loss  +(torch.square(y_decoded.variance) / torch.square(y_decoded.mean))
   
    return torch.sum(torch.distributions.kl.kl_divergence(z_decoded, prior_z) - log_norm(y_decoded.sample(), y_decoded.mean, torch.sqrt(y_decoded.variance)) )

In [ ]:
class VAEDataset(Dataset):
    def __init__(self, x_df,t_df, y_df):
        # for batch, (x, y) in enumerate(dataloader):
        self.x = x_df
        self.t = t_df
        self.y = y_df 
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx): 
        return self.x[idx], self.t[idx], self.y[idx]

In [ ]:
 def vae_dataset_prep(df, t_index):
    label_column = df.columns[-1]
    treatment_column = df.columns[t_index]
    train, test = train_test_split(df, test_size=0.1)

    x_train = train.loc[:, ~train.columns.isin([treatment_column, label_column])]
    t_train = train.loc[:, train.columns == treatment_column]
    y_train = train.loc[:, train.columns == label_column]

    # x_test = test.loc[:, test.columns != label_column]
    # y_test = test.loc[:, test.columns == label_column]
    return torch.from_numpy(x_train.values).float(), torch.from_numpy(t_train.values).float(), torch.from_numpy(y_train.values).float()

In [ ]:
def vae_data_loader_pref(df, t_index):
    x_train, t_train, y_train = vae_dataset_prep(df, t_index)

    train_dataset = VAEDataset(x_train,t_train, y_train)
    trainloader = DataLoader(train_dataset, batch_size=16)

    return trainloader, len(train_dataset[0][0])

In [ ]:
def train_vae(train_loader, n_dimensions):
    vae = VAE(n_dimensions=n_dimensions)
    vae = vae.train()  # set training mode
    loss_func = T.nn.MSELoss()  # mean squared error
    # loss = torch.sqrt(criterion(x, y))
    prior_z = torch.distributions.Normal(torch.zeros(n_dimensions), torch.eye(1))

    optimizer = T.optim.Adam(vae.parameters(), lr=0.0001)
  
    for batch, (x, t, y) in enumerate(train_loader):
        optimizer.zero_grad()
        z_decoded, y_decoded, t_encoded, x_encoded = vae(x, t)
        loss_obj = elbo_loss(y_decoded, y, z_decoded, prior_z)  # Root mean squared error
        loss_obj.backward()
        optimizer.step()

    maximum_value = train_loader.dataset.t.max()
    minimum_value = train_loader.dataset.t.min()
    treatment_range = maximum_value - minimum_value
    # Implement a function here to determine how to intervene to T
    # trainloader.dataset.t Test this
    total_effect = 0
    for batch, (x, t, y) in enumerate(train_loader):
        batch_size_ = x.shape[0]
        _, y_decoded_max, _, _ = vae(x, torch.stack([maximum_value] * batch_size_).reshape(batch_size_,1))
        _, y_decoded_min, _, _ = vae(x, torch.stack([minimum_value] * batch_size_).reshape(batch_size_,1))
        
        effect = torch.sum(torch.abs((y_decoded_max.mean - y_decoded_min.mean)))/ batch_size_
        total_effect += effect.item()

    return total_effect / len(train_loader)

In [ ]:
# train_loader, n_dimensions = vae_data_loader_pref(boston_df, t_index=0)

In [ ]:
def calculate_treatment_effect(df):
    treatment_effect_dict = {}
    n_intervensions = len(df.columns)-1
    for intervention_index in range(n_intervensions):
        train_loader, n_dimensions = vae_data_loader_pref(df, t_index=intervention_index)
        ate_ = train_vae(train_loader, n_dimensions)
        treatment_effect_dict[intervention_index] = ate_
    return treatment_effect_dict

In [ ]:
boston_effect_dict = calculate_treatment_effect(boston_df)
energy_effect_dict = calculate_treatment_effect(energy)
concrete_effect_dict = calculate_treatment_effect(concrete_)

In [ ]:
 def causal_dataset_prep(df, selected_indexes):
    label_column = df.columns[-1]
    selected_column = []
    for c_index in selected_indexes:
        selected_column.append(df.columns[c_index])
    train, test = train_test_split(df, test_size=0.1)

    x_train = train.loc[:, train.columns.isin(selected_column)]
    y_train = train.loc[:, train.columns == label_column]

    x_test = test.loc[:, test.columns.isin(selected_column)]
    y_test = test.loc[:, test.columns == label_column]
    return torch.from_numpy(x_train.values).float(), torch.from_numpy(y_train.values).float(), torch.from_numpy(x_test.values).float(), torch.from_numpy(y_test.values).float()

In [ ]:
def index_selector(score_dict):
    sorted_scores = sorted(score_dict.values(), reverse=True)
    best_indexes =[]
    for i in range(3):
        for column_index, score in score_dict.items():
            if score == sorted_scores[i]:
                best_indexes.append(column_index)
    return best_indexes


In [ ]:
def causal_data_loader_pref(df, score_dict):
    selected_indexes = index_selector(score_dict)
    x_train, y_train, x_test, y_test = causal_dataset_prep(df, selected_indexes)

    train_dataset = RegressionDataset(x_train, y_train)
    trainloader = DataLoader(train_dataset, batch_size=16)
    test_dataset = RegressionDataset(x_test, y_test)
    testloader = DataLoader(test_dataset, batch_size=16)
    return trainloader, testloader, len(train_dataset[0][0])

In [ ]:
def causal_linear_regressor_wrapper(df, score_dict):
    train_loader, test_loader, n_dimensions = causal_data_loader_pref(df, score_dict)
    eval_loss = train_linear_regressor(train_loader, test_loader, n_dimensions)
    return eval_loss

In [ ]:
boston_loss = causal_linear_regressor_wrapper(boston_df, boston_effect_dict)
print('Boston Causal RMSE Loss: {}'.format(boston_loss))
energy_loss = causal_linear_regressor_wrapper(energy, energy_effect_dict)
print('Energy Causal RMSE Loss: {}'.format(energy_loss))
concrete_loss = causal_linear_regressor_wrapper(concrete_, concrete_effect_dict)
print('Concrete Causal RMSE Loss: {}'.format(concrete_loss))


Boston Causal RMSE Loss: 13.546119928359985
Energy Causal RMSE Loss: 23.12357940673828
Concrete Causal RMSE Loss: 16.555611065455846
